#### Haiku Generator
Description of solution will go here

Dataset can be found on Kaggle at: https://www.kaggle.com/datasets/marzenah/azlyrics-recorded-songs-with-lyrics 

In [ ]:
import pandas as pd
import re
import syllapy
import nltk
from nltk.corpus import words

In [ ]:
# Read the dataset and display the rows and columns
data = pd.read_csv('h_artists_songs.csv') 

data.head()

In [ ]:
# Load only the Lyrics column and display the new dataframe
df = pd.DataFrame(data['Lyrics'])

df

In [ ]:
# Convert the lyrics dataframe into a list
lyrics = df['Lyrics'].tolist()

# Clean every lyric from interpunction and other non-text characters
lyrics = [re.sub(r"[^\w\s]", ' ', lyric) for lyric in lyrics]

# Remove the word "Romanized" as it is not part of the lyrics
lyrics = [re.sub('Romanized', ' ', text) for text in lyrics]

lyrics[:20]

In [ ]:
# Download the English word corpus from NLTK
nltk.download('words')

# Create an English dictionary object
english_words = set(words.words())

# Function to filter out non-English words

# TODO: keep lyrics seperated by line
lyrics = [word for line in lyrics for word in line.split() if word.lower() in english_words]
# lyrics = [word for line in lyrics for word in line.split() if word.lower() not in english_words]
# lyrics = [re.sub(lyrics, ' ', word) for word.lower() not in english_words]

lyrics[:20]

In [ ]:
def separate_syllables(text):

    words = text.split()
    separated_text = []

    for word in words:

        # Use syllapy to count the amount of syllables in a word
        syllables = syllapy.count(word)
        
        separated_text.append(syllables)
    
    # Return number of syllables for every word in lyrics
    return separated_text



text = lyrics[0]
separated_text = separate_syllables(text)
print(separated_text)

# for lyric in lyrics:
#     text = lyric
        
#     separated_text = separate_syllables(text)
#     print(separated_text)


In [ ]:
# Functions to check for 5, 7, 5 patterns in lyrics

def count_syllables(words):
    # Return the sum of counted syllables for each individual word
    return sum(syllapy.count(word) for word in words)

def recognize_575_pattern(text):
    # Split text into words by separating on spaces
    words = text.split()

    # Initialize syllable count list
    syllable_counts = []
    count = 0

    # Make sure full words are used to count syllables instead of parts of words
    for word in words:
        # print(word)
        count += syllapy.count(word)
        if count == 5 or count == 12:  # Checks for the fifth and twelfth syllable and appends to syllable_counts if true
            syllable_counts.append(count)

    if count == 17:
        syllable_counts.append(count)  # Append the syllable count for the last line
        print(syllable_counts)
        count = 0
    
    return syllable_counts == [5, 12, 17] # Return if the counts are 5, 12, 17 in that order (5-7-5 pattern)

# test haiku (in 5,7,5 format)
haiku = "new teaching format new better teaching format new teaching format"

#TODO: loop through lyrics instead of using a test haiku

for lyric in lyrics:
    text = lyric

    if recognize_575_pattern(text):
        print(f"This lyric is a haiku: \n {lyric}") #TODO: formatting into 5-7-5 syllable lines 